In [65]:
import json
import pandas as pd

In [66]:
data = 'C:/Users/cates/data/diary.json'
dt = None
with open(data, mode='r') as file:
    dt = json.load(file)

In [67]:
dt_converted = {}
date_appearance_counter = []

for index in dt[0:5]:
    print(index)
    convert = dict(index)
    new_key = convert['date'] #grabs the date to use as key in new dict object
    date_appearance_counter.append(new_key) #add date to list to count number of appearances easily
    counter = date_appearance_counter.count(new_key) #count appearances
    key = new_key + '_' + str(counter) #concatenate date + appearances for new dict
    dt_converted[key] = convert 

print(dt_converted)

{'deleted_at': '', 'date': '2022-12-04', 'food': '14" Pepperoni Pizza', 'brand': '', 'servings': 2, 'servingsUnit': 'slice', 'calories': 596, 'protein': '26.64g', 'carbs': '67.96g', 'fat': '24.26g', 'nutrients': {'created_at': '2022-12-04 21:55:16', 'updated_at': '2022-12-04 21:55:16', 'sourceVersion': 2, 'saturatedFat': '10.52g', 'monoFat': '7.46g', 'polyFat': '4.12g', 'sugar': '8.22g', 'fiber': '3.2g', 'sodium': '1366mg', 'cholesterol': '58mg', 'potassium': '372mg', 'vitaminA': '120%', 'calcium': '336%', 'iron': '4.76%'}}
{'deleted_at': '', 'date': '2022-12-04', 'food': 'Cheeseburger', 'brand': 'Five Guys', 'servings': 2, 'servingsUnit': 'serving', 'calories': 1960, 'protein': '94g', 'carbs': '80g', 'fat': '110g', 'nutrients': {'created_at': '2022-12-04 23:32:36', 'updated_at': '2022-12-04 23:32:36', 'sourceVersion': 2, 'saturatedFat': '53g', 'sugar': '18g', 'fiber': '4g', 'sodium': '2100mg', 'cholesterol': '330mg'}}
{'deleted_at': '', 'date': '2022-12-04', 'food': 'Milkshake', 'bran

In [68]:
df = pd.DataFrame(dt)
print(df.head())

#convert df columns to vectors to go fast
foods = df['food']
protein = df['protein']
carbs = df['carbs']
fat = df['fat']
calories = df['calories']
unit = df['servingsUnit']
amount = df['servings']

#fill blanks in each list with 0
protein = [str(0) if i == '' else i for i in protein]
carbs = [str(0) if i == '' else i for i in carbs]
fat = [str(0) if i == '' else i for i in fat]
calories = [str(0) if i == '' else i for i in calories]

#convert integer record to string in food list
foods = [str('Deleted') if str(i).isnumeric() == True else i for i in foods]
foods = [i.lower() for i in foods]

  deleted_at        date                 food      brand  servings  \
0             2022-12-04  14" Pepperoni Pizza                  2.0   
1             2022-12-04         Cheeseburger  Five Guys       2.0   
2             2022-12-04            Milkshake  Five Guys       1.0   
3             2022-12-05          Raspberries                 45.0   
4             2022-12-05   Organic Brown Rice        365     212.0   

  servingsUnit  calories protein   carbs     fat  \
0        slice    596.00  26.64g  67.96g  24.26g   
1      serving   1960.00     94g     80g    110g   
2      serving    670.00     13g     84g     32g   
3            g     23.36   0.54g   5.36g   0.32g   
4            g    298.50   7.42g  64.24g   2.33g   

                                           nutrients  
0  {'created_at': '2022-12-04 21:55:16', 'updated...  
1  {'created_at': '2022-12-04 23:32:36', 'updated...  
2  {'created_at': '2022-12-04 23:32:57', 'updated...  
3  {'created_at': '2022-12-05 01:53:21', 'upda

In [69]:
target_protein = 195.0
target_carbs = 408.0
target_fat = 96.0
target_cals = (target_protein * 5) + (target_carbs * 4) + (target_fat * 9)
current_protein = 190.0
current_carbs = 375.0
current_fat = 80.0
current_cals = (current_protein * 5) + (current_carbs * 4) + (current_fat * 9)

calorie_balance = target_cals - current_cals 
protein_balance = target_protein - current_protein
carb_balance = target_carbs - current_carbs
fat_balance = target_fat - current_fat

In [70]:
ingredients = [
    'milk',
    'waffle',
    'protein',
    'yogurt',
    'granola',
    'chicken',
    'banana',
    'peanut butter',
    'syrup',
    'egg',
    'bread',
    'rice',
    'oat'    
]

In [71]:
counter = []

for i, ing in enumerate(ingredients):
    hits = 0
    for m in foods:
        if ing in m:
            hits += 1
        else:
            continue
    counter.append(hits)

usage = list(zip(ingredients,counter))
print(usage)

[('milk', 578), ('waffle', 59), ('protein', 126), ('yogurt', 129), ('granola', 83), ('chicken', 310), ('banana', 184), ('peanut butter', 93), ('syrup', 27), ('egg', 168), ('bread', 74), ('rice', 185), ('oat', 114)]


In [72]:
foods_available = []
valid_meals = {}

def identify_options(protein_balance, carb_balance, fat_balance):
    for i, food in enumerate(foods):
        for item in ingredients:
            if item in food:
                foods_available.append(food)
            else:
                continue
        carb_update = carb_balance - float(carbs[i].replace('g','')) 
        protein_update = protein_balance - float(protein[i].replace('g',''))
        fat_update = fat_balance - float(fat[i].replace('g',''))
        calories_update = calorie_balance - float(calories[i])
        if food in foods_available:
            if carb_update > 0.0 and protein_update > 0.0 and fat_update > 0.0:   
                valid_meals[food] = {'carb': carbs[i], 'fat': fat[i], 'protein': protein[i], 'serving': amount[i], 'unit': unit[i], 
                                     'macros_left': {'c': carb_update, 'f': fat_update, 'p': protein_update, 'cal': calories_update}}
            else: 
                continue
        else:
            continue
    return valid_meals
identify_options(protein_balance, carb_balance, fat_balance)

{'waffle': {'carb': '15g',
  'fat': '3g',
  'protein': '2g',
  'serving': 1.0,
  'unit': 'serving',
  'macros_left': {'c': 18.0, 'f': 13.0, 'p': 3.0, 'cal': 206.0}},
 'organic creamy peanut butter unsweetened': {'carb': '1.88g',
  'fat': '5.63g',
  'protein': '2.19g',
  'serving': 10.0,
  'unit': 'g',
  'macros_left': {'c': 31.12,
   'f': 10.370000000000001,
   'p': 2.81,
   'cal': 238.5}},
 'coconut cashew granola': {'carb': '30.73g',
  'fat': '15.77g',
  'protein': '4.16g',
  'serving': 54.0,
  'unit': 'g',
  'macros_left': {'c': 2.2699999999999996,
   'f': 0.23000000000000043,
   'p': 0.8399999999999999,
   'cal': 26.839999999999975}},
 'skim milk': {'carb': '5.5g',
  'fat': '0',
  'protein': '4g',
  'serving': 0.5,
  'unit': 'serving',
  'macros_left': {'c': 27.5, 'f': 16.0, 'p': 1.0, 'cal': 261.0}},
 'organic multigrain waffles': {'carb': '30g',
  'fat': '6g',
  'protein': '4g',
  'serving': 1.0,
  'unit': 'serving',
  'macros_left': {'c': 3.0, 'f': 10.0, 'p': 1.0, 'cal': 111.0}},

In [73]:
#could have added this section back to the function, created another loop to iterate through multiple choice paths, etc. 

food_names = []
food_values = []
for food in valid_meals:
    food_names.append(food)
    food_values.append(valid_meals[food]['macros_left']['cal'])

final_list = list(zip(food_names,food_values))
final_list.sort(key=lambda x: x[1]) #sorts list by second index

print('the best food option based on my 2023 history, accounting for my macros today and overall targets would be: \n' + str(final_list[0][0]) + '\n' + 
      str(valid_meals[final_list[0][0]]))

the best food option based on my 2023 history, accounting for my macros today and overall targets would be: 
coconut cashew granola
{'carb': '30.73g', 'fat': '15.77g', 'protein': '4.16g', 'serving': 54.0, 'unit': 'g', 'macros_left': {'c': 2.2699999999999996, 'f': 0.23000000000000043, 'p': 0.8399999999999999, 'cal': 26.839999999999975}}
